In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/diabetes (1).csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
X = df.drop('Outcome' , axis='columns')
Y = df.Outcome

In [24]:
Y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [25]:
268/500

0.536

###Train Test Split

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)
x_scaled[:3]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [19]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(x_scaled,Y , stratify=Y , random_state=10)

In [20]:
X_train.shape

(576, 8)

In [21]:
X_test.shape

(192, 8)

In [22]:
192/576 

0.3333333333333333

In [26]:
Y_train.value_counts()

0    375
1    201
Name: Outcome, dtype: int64

In [27]:
201/375

0.536

We have the Same Ratio as the Y , and that determine the good sampling in the splitting data

###Train using stand alone model


In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores = cross_val_score(DecisionTreeClassifier() , X,Y, cv=5)
scores

array([0.66883117, 0.66233766, 0.68181818, 0.78431373, 0.73202614])

In [33]:
scores.mean()

0.7058653764536117

###Train using Bagging


In [30]:
from sklearn.ensemble import BaggingClassifier
bag_m = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
bag_m.fit(X_train,Y_train)
bag_m.oob_score_

0.7534722222222222

In [36]:
bag_m.score(X_test,Y_test)

0.7760416666666666

By doing the regular Scoring for X and Y test and we see improvement , 77% versus standalone model giving us 71% (scores)

In [37]:
bag_m = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
scoree = cross_val_score(bag_m , X,Y,cv=5)
scoree.mean()

0.7578728461081402

We can see some improvement in test score with bagging classifier as compared to a standalone classifier


By creating the same bagging model and use the Cross validation Score and we find that the base model give us 70,586% Accuracy , But the bagged model give us 75,787% Accuracy. \
So for Unstable classifier like Decision Tree Bagging helps .
if you have classifier unstable and sometimes your dataset are so many nun values and a high variance model it make sense to use **BaggingClassifier**


###Train using Random Forest


In [39]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, Y, cv=5)
scores.mean()

0.7487649605296663